# Valid Dataset

Here we only keep admissions which are the first admissions of some patients and are of patients >= 15 years old.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import shutil

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

%matplotlib inline

In [2]:
res_dir = '/media/data/mimic/res'

In [8]:
_adm_first = np.load(f'{res_dir}/admission_first_ids.npy', allow_pickle=True).tolist()
admission_first_ids_list = _adm_first['admission_ids']
print(len(admission_first_ids_list))

admission_ids = [re.match(r'adm\-(\d+)\.npy', x) for x in os.listdir('/media/data/mimic/admdata/')]
admission_ids = [int(x.group(1)) for x in admission_ids if x is not None]
print(len(admission_ids), admission_ids[:10])

admission_first_ids_set = set(admission_first_ids_list)
print(len(admission_first_ids_set))
admission_first_ids = [x for x in admission_ids if x in admission_first_ids_set]
print(len(admission_first_ids), admission_first_ids[:10])

46520
58576 [125195, 121731, 107612, 130785, 182359, 185381, 132654, 120303, 142716, 102298]
46520
46230 [121731, 107612, 130785, 182359, 185381, 132654, 142716, 102298, 134562, 145214]


In [19]:
170719 in admission_first_ids

True

In [20]:
TARGETDIR = '/media/data/mimic/admdata_valid'
if not os.path.exists(TARGETDIR):
    os.makedirs(TARGETDIR)

## Store valid data

We store all datafiles belonging to valid admission ids in a specific folder (../../Data/admdata_valid)

In [22]:
def copy_valid_admissions(aid):
    if aid in admission_first_ids_set:
        res = np.load(os.path.join('/media/data/mimic/admdata', 'adm-%.6d.npy' % aid), allow_pickle=_pickle=True).tolist()
        general = res['general']
        age = general[2]
        if age >= 15 * 365.25:
            np.save(os.path.join(TARGETDIR, 'adm-%.6d.npy' % aid), res)
        
p = Pool()
p.map(copy_valid_admissions, admission_ids)
p.close()
p.join()